In [ ]:
!pip install onnx onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.8 MB/s eta 0:00:00


In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim

# Datos de ejemplo extendidos
data = [
    ("hola", "¡Hola! ¿Como estas?"),
    ("¿como estas?", "Estoy bien, gracias por preguntar."),
    ("adios", "¡Adios! Cuidate."),
    ("¿cual es tu nombre?", "Soy un chatbot simple."),
    ("¿que haces?", "Estoy aqui para ayudarte."),
    ("¿que es python?", "Python es un lenguaje de programacion popular."),
    ("cuentame un chiste", "¿Por que el libro de matematicas estaba triste? Porque tenia muchos problemas."),
    ("¿que dia es hoy?", "No tengo un calendario, pero puedo ayudarte con otras cosas."),
    ("gracias", "De nada, estoy para servirte."),
    ("¿puedes ayudarme?", "¡Claro! Dime en que necesitas ayuda."),
]

# Preprocesamiento del texto
def tokenize(sentence):
    return sentence.lower().split()

def build_vocab(data):
    words = set()
    for question, _ in data:
        words.update(tokenize(question))
    return {word: idx for idx, word in enumerate(words)}

vocab = build_vocab(data)
vocab_size = len(vocab)

# Convertir texto a tensores
def text_to_tensor(text, vocab):
    tokens = tokenize(text)
    indices = [vocab.get(word, 0) for word in tokens]
    return torch.tensor(indices, dtype=torch.float32)

# Red neuronal
class ChatbotModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_classes):
        super(ChatbotModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.fc1 = nn.Linear(embed_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.embedding(x.long())
        x = x.mean(dim=1)
        x = torch.relu(self.fc1(x))
        out = self.fc2(x)
        return out

# Parámetros del modelo
embed_size = 8
hidden_size = 16
num_classes = len(data)
model = ChatbotModel(vocab_size, embed_size, hidden_size, num_classes)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Entrenamiento del modelo
epochs = 500
for epoch in range(epochs):
    total_loss = 0
    for idx, (question, _) in enumerate(data):
        x = text_to_tensor(question, vocab).unsqueeze(0)
        y = torch.tensor([idx])
        optimizer.zero_grad()
        output = model(x)
        loss = loss_fn(output, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    if (epoch + 1) % 50 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(data):.4f}")

# Exportar modelo a ONNX
example_input = text_to_tensor("hola", vocab).unsqueeze(0)
model = model.float()
torch.onnx.export(
    model,
    example_input,
    "chatbot_model.onnx",
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={'input': {0: 'batch_size', 1: 'sequence_length'}, 'output': {0: 'batch_size'}},
    opset_version=11
)

print("Modelo exportado como chatbot_model.onnx")


Epoch [50/500], Loss: 0.0035
Epoch [100/500], Loss: 0.0007
Epoch [150/500], Loss: 0.0003
Epoch [200/500], Loss: 0.0002
Epoch [250/500], Loss: 0.0001
Epoch [300/500], Loss: 0.0001
Epoch [350/500], Loss: 0.0000
Epoch [400/500], Loss: 0.0000
Epoch [450/500], Loss: 0.0000
Epoch [500/500], Loss: 0.0000
Modelo exportado como chatbot_model.onnx
